In [1]:
from sklearn.datasets import load_iris

iris = load_iris()

# 特征选择(Feature-Selection)

## 1. Filter Method

### 1.1 方差(variance)选择法
#### 方差：在概率学里，方差是衡量随机变量或一组数据的离散程度，值越大，离散程度越大。
那么在数据分析/机器学习中，某个特征的数据的方差值越小，则说明这个值的离散程度很小，变化程度小，因此有很多大概率对target的影响没那么大，因此可以选择舍弃此特征。相反，如果方差值越大，则说明特征值在不同样本里的变化程度也大，有很大概率对target有很大的详细程度，应保留。

In [2]:
from sklearn.feature_selection import VarianceThreshold

# 设定阈值threshold,方差大于阈值，则选择
VT = VarianceThreshold(threshold=3)
vt_data = VT.fit_transform(iris.data)

从下面值可以看出，特征选择之后，只保留了一个特征

In [3]:
iris.data.shape, vt_data.shape

((150, 4), (150, 1))

### 1.2 相关系数法(Pearson)
#### 相关系数法：计算各个特征值对target值的相关系数的pearson值，局限性是pearson相关系数计算的结果是线性关系，公式为：
$$ r=\frac{N\sum x_iy_i - \sum x_i \sum y_i}{\sqrt{N\sum x_i^2-(\sum x_i)^2} \sqrt{N\sum y_i^2-(\sum yi)^2}} $$


In [4]:
from sklearn.feature_selection import SelectKBest  # 移除得分前K名以外的特征，即只保留TopK的特征
from scipy.stats import pearsonr   # scipy.stats.pearsonr库里计算的pearson相关系数
import numpy as np

In [5]:
# 创建一个计算多个特征的pearson系数的函数
def multiFeature_pearsonr(X, y):
    '''
    params: X: 输入特征
            y: 输入标签
    '''
    scores, p_values = [], []
    for ret in map(lambda x:pearsonr(x, y), X.T):
        scores.append(abs(ret[0]))
        p_values.append(ret[1])
    
    return (np.array(scores), np.array(p_values))

sk = SelectKBest(score_func=multiFeature_pearsonr, k=2)  # 选择top2
x_pearson = sk.fit_transform(iris.data, iris.target)   # 会自动调用multiFeature_pearsonr(X, y)函数

In [6]:
x_pearson.shape

(150, 2)

### 1.3 卡方检验方法(chi-square test method)
#### 卡方检验：该方法就是检验定性自变量对因变量的相关性。假设自变量有N种取值，因变量有M种取值，考虑自变量等于i且因变量等于j的样本频数的观察值与期望的差距，构建统计量：
$$ x^2 = \sum{\frac{(A-E)^2}{E}}=\sum_{i=1}^{k}\frac{(A_i-E_i)^2}{E_i}=\sum_{i=1}^{k}\frac{(A_i-np_i)^2}{np_i} $$

In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

sk_chi2 = SelectKBest(score_func=chi2, k=2)
chi2_data = sk_chi2.fit_transform(iris.data, iris.target)

In [8]:
chi2_data.shape

(150, 2)

### 1.4 互信息法
#### 互信息法：也是检验自变量与因变量的相关性的，互信息计算公式为：
$$ I(X;Y)=\sum_{x\in{X}} \sum_{y\in{Y}}p(x,y)\log{p(x,y)/p(x)p(y)}$$

In [9]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

sk_mi = SelectKBest(mutual_info_classif, k=2)
mi_data = sk_mi.fit_transform(iris.data, iris.target)

In [10]:
mi_data.shape

(150, 2)